In [1]:
import os
import glob
import findspark
findspark.init()
import time
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StringType, IntegerType, StructField, ArrayType, DoubleType, DateType
from pyspark.sql.functions import col
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
año2019 = spark.read.csv("datos_aire/aire2019.csv",header=True, inferSchema=True, sep=";")
año2020 = spark.read.csv("datos_aire/aire2020.csv",header=True, inferSchema=True, sep=";")
año2021 = spark.read.csv("datos_aire/aire2021.csv",header=True, inferSchema=True, sep=";")
año2022 = spark.read.csv("datos_aire/aire2022.csv",header=True, inferSchema=True, sep=";")
añosAire = año2019.union(año2020).union(año2021).union(año2022)
añosAire = añosAire.drop("v01","v02","v03","v04","v05","v06","v07","v08","v09","v10"
                         ,"v11","v12","v13","v14","v15","v16","v17","v18","v19","v20"
                         ,"v21","v22","v23","v24")

In [6]:
cols = ["h01","h02","h03","h04","h05","h06","h07","h08","h09","h10"
       ,"h11","h12","h13","h14","h15","h16","h17","h18","h19","h20"
       ,"h21","h22","h23","h24"]
for col_name in cols:
    añosAire = añosAire.withColumn(col_name, regexp_replace(col_name, ',', '.'))
    añosAire = añosAire.withColumn(col_name,añosAire.h01.cast('float'))

añosAire = añosAire.withColumn('h01', regexp_replace('h01', ',', '.'))
añosAire = añosAire.withColumn("h01",añosAire.h01.cast('float'))
añosAire = añosAire.withColumn("media_24h", (col("h01")+col("h02")+ col("h03")+ col("h04")+ col("h05")
                        + col("h06")+ col("h07")+ col("h08")+ col("h09")+ col("h10")
                        + col("h11")+ col("h12")+ col("h13")+ col("h14")+ col("h15")
                        + col("h16")+ col("h17")+ col("h18")+ col("h19")+ col("h20")
                        + col("h21")+ col("h22")+ col("h23")+ col("h24")) / lit(24))
añosAire = añosAire.drop("h01","h02","h03","h04","h05","h06","h07","h08","h09","h10"
                         ,"h11","h12","h13","h14","h15","h16","h17","h18","h19","h20"
                         ,"h21","h22","h23","h24")

añosAire = añosAire.withColumn('ano', concat(col("ano"),lit("-"),col("mes")).alias('fecha'))
añosAire = añosAire.withColumnRenamed("ano","Fecha")
añosAire = añosAire.drop("mes","dia","estacion","punto_muestreo","provincia","municipio")


In [4]:
mediaMagnitud = añosAire.groupby("Fecha", "magnitud").mean("media_24h")


In [5]:
#RANGO 1 - Meses de Diciembre 2019 y Enero / Febrero 2020
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2019-12") 
                              | (mediaMagnitud.Fecha == "2020-1") 
                              | (mediaMagnitud.Fecha == "2020-2"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 1"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango1 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango1.show()


+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono| Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|Rango 1| 0.6214547668700391|1.2544592030360529|   17.447504702312354|  25.817908994947985|  52.71089663853794|   1.4538789988374354|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+



In [6]:
#RANGO 2 - Meses de Marzo / Abril / Mayo 2020
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2020-3") 
                              | (mediaMagnitud.Fecha == "2020-4") 
                              | (mediaMagnitud.Fecha == "2020-5"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 2"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango2 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango2.show()

+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono|Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+
|Rango 2| 0.4480774009327873|1.297079337401918|   1.6833746671314687|   8.977033506095728| 11.979057566422796|    1.360752566854871|
+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+



In [7]:
#RANGO 3 - Meses de Junio / Julio / Agosto / Septiembre / Octubre 2020
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2020-6") 
                              | (mediaMagnitud.Fecha == "2020-7")
                              | (mediaMagnitud.Fecha == "2020-8")
                              | (mediaMagnitud.Fecha == "2020-9")
                              | (mediaMagnitud.Fecha == "2020-10"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 3"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango3 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango3.show()

+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono| Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|Rango 3| 0.4411144319576782|1.7797453961191447|   1.6842190452248709|  12.044947789110415| 15.017845546801007|    1.277924999795841|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+



In [8]:
#RANGO 4 - Meses de Noviembre / Diciembre 2020 y Enero / Febrero / Marzo / Abril 2021
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2020-11") 
                              | (mediaMagnitud.Fecha == "2020-12")
                              | (mediaMagnitud.Fecha == "2021-1")
                              | (mediaMagnitud.Fecha == "2021-2")
                              | (mediaMagnitud.Fecha == "2021-3")
                              | (mediaMagnitud.Fecha == "2021-4"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 4"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango4 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango4.show()

+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono|Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+
|Rango 4| 0.4237501363809801|1.551086677689721|    6.378229872602186|  16.244671863518803|  26.31054199660144|   1.3512811053021168|
+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+



In [9]:
#RANGO 5 - Meses de Mayo / Junio / Julio / Agosto / Septiembre / Octubre / Noviembre  2021
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2021-5") 
                              | (mediaMagnitud.Fecha == "2021-6")
                              | (mediaMagnitud.Fecha == "2021-7")
                              | (mediaMagnitud.Fecha == "2021-8")
                              | (mediaMagnitud.Fecha == "2021-9")
                              | (mediaMagnitud.Fecha == "2021-10")
                              | (mediaMagnitud.Fecha == "2021-11"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 5"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango5 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango5.show()

+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono|Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+
|Rango 5| 0.4007337832940635|1.811987933406207|   2.4278754420636295|  13.664516214062653|  17.73255980932964|   1.3165925332651371|
+-------+-------------------+-----------------+---------------------+--------------------+-------------------+---------------------+



In [10]:
#RANGO 6 - Meses de Diciembre 2021 y Enero / Febrero / Marzo / Abril / Mayo / Junio 2022
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2021-12") 
                              | (mediaMagnitud.Fecha == "2022-1")
                              | (mediaMagnitud.Fecha == "2022-2")
                              | (mediaMagnitud.Fecha == "2022-3")
                              | (mediaMagnitud.Fecha == "2022-4")
                              | (mediaMagnitud.Fecha == "2022-5")
                              | (mediaMagnitud.Fecha == "2022-6"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 6"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango6 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango6.show()

+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono| Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|Rango 6| 0.3857371033357376|1.7743372196938616|    5.958511496045564|   16.17012255988493|   25.6088218111856|   1.3433278515414386|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+



In [11]:
#RANGO 7 - Meses de Julio / Agosto / Septiembre / Octubre / Noviembre / Diciembre 2022
def media_tramo1(numMagnitud,nomMagnitud):
    fase1 = mediaMagnitud.where(mediaMagnitud.magnitud == numMagnitud)
    fase1 = fase1.where((mediaMagnitud.Fecha == "2022-7") 
                              | (mediaMagnitud.Fecha == "2022-8")
                              | (mediaMagnitud.Fecha == "2022-9")
                              | (mediaMagnitud.Fecha == "2022-10")
                              | (mediaMagnitud.Fecha == "2022-11")
                              | (mediaMagnitud.Fecha == "2022-12"))
    fase1 = fase1.groupby("magnitud").mean("avg(media_24h)")
    fase1 = fase1.withColumnRenamed("avg(avg(media_24h))",str(nomMagnitud))
    fase1 = fase1.withColumnRenamed("magnitud","RANGO")
    fase1 = fase1.filter(fase1.RANGO == str(numMagnitud)).withColumn('RANGO', lit("Rango 7"))
    return fase1

media1 = media_tramo1(6,"Monóxido de Carbono")
media2 = media_tramo1(1,"Dioxido de Azufre")
media3 = media_tramo1(7,"Monóxido de Nitrógeno")
media4 = media_tramo1(8,"Dióxido de Nitrógeno")
media5 = media_tramo1(12,"Óxidos de Nitrógeno")
media6 = media_tramo1(42,"Hidrocarburos Totales")
rango7 = media1.join(media2, ['RANGO']).join(media3, ['RANGO']).join(media4, ['RANGO']).join(media5, ['RANGO']).join(media6, ['RANGO'])
rango7.show()

+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono| Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|Rango 7|  0.471220891477129|1.9374143786965918|     4.32452783275945|  15.063774332665444|  21.98986947761495|   1.3247176515595898|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+



In [12]:
rangos_medias = rango1.union(rango2).union(rango3).union(rango4).union(rango5).union(rango6).union(rango7)

In [13]:
rangos_medias.show()

+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|  RANGO|Monóxido de Carbono| Dioxido de Azufre|Monóxido de Nitrógeno|Dióxido de Nitrógeno|Óxidos de Nitrógeno|Hidrocarburos Totales|
+-------+-------------------+------------------+---------------------+--------------------+-------------------+---------------------+
|Rango 1| 0.6214547668700391|1.2544592030360529|   17.447504702312354|  25.817908994947985|  52.71089663853794|   1.4538789988374354|
|Rango 2| 0.4480774009327873| 1.297079337401918|   1.6833746671314687|   8.977033506095728| 11.979057566422796|    1.360752566854871|
|Rango 3| 0.4411144319576782|1.7797453961191447|   1.6842190452248709|  12.044947789110415| 15.017845546801007|    1.277924999795841|
|Rango 4| 0.4237501363809801| 1.551086677689721|    6.378229872602186|  16.244671863518803|  26.31054199660144|   1.3512811053021168|
|Rango 5| 0.4007337832940635| 1.811987933406207|   2.427875442